<a href="https://colab.research.google.com/github/ahmed98Osama/YT-Download-Gofile-Upload/blob/main/Download_YouTube_Videos_Using_yt_dlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install yt-dlp

Install or upgrade the `yt-dlp` package (required for downloading).

In [5]:
%pip install yt-dlp

## 2. Simple single-video download & Upload to Gofile (curl)

Download one YouTube video in best quality (interactive: prompts for URL and save path), then optionally upload a file to Gofile.io via `curl`. Adjust the curl file path to match your downloaded file.

In [ ]:
import yt_dlp  # pyright: ignore[reportMissingImports]

def download_youtube_video(video_url, save_path="downloads"):
    """
    Downloads a YouTube video in the best available video and audio quality.

    Args:
        video_url (str): URL of the YouTube video.
        save_path (str): Directory to save the downloaded video.
    """
    try:
        # Options for yt-dlp
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',  # Download best video and audio and merge
            'outtmpl': f'{save_path}/%(title)s.%(ext)s',  # Save file format
            'merge_output_format': 'mp4',  # Merge video and audio into MP4 format
        }

        # Downloading the video
        print("Downloading...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        print(f"Video downloaded successfully and saved in: {save_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # Input YouTube video URL
    video_url = input("Enter the YouTube video URL: ").strip()

    # Input save directory (default is 'downloads')
    save_path = input("Enter the save path (or press Enter for default 'downloads'): ").strip() or "downloads"

    # Call the function to download the video
    download_youtube_video(video_url, save_path)

# --- Optional: upload a file to Gofile.io (adjust file path as needed) ---
!curl -F "file=@/content/downloads/YOUR_FILE.mp4" https://upload.gofile.io/uploadfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.6 MB/s eta 0:00:00
Enter the YouTube video URL: https://www.youtube.com/watch?v=2wBYW8hnO4A
Enter the save path (or press Enter for default 'downloads'): 
Downloading...
[youtube] Extracting URL: https://www.youtube.com/watch?v=2wBYW8hnO4A
[youtube] 2wBYW8hnO4A: Downloading webpage


[youtube] 2wBYW8hnO4A: Downloading android sdkless player API JSON
[youtube] 2wBYW8hnO4A: Downloading web safari player API JSON


[youtube] 2wBYW8hnO4A: Downloading m3u8 information


[info] 2wBYW8hnO4A: Downloading 1 format(s): 299+251
[download] Destination: downloads/Lost Judgment - All Cutscenes Full Movie (2021).f299.mp4
[download] 100% of   22.95GiB in 00:08:01 at 48.78MiB/s  
[download] Destination: downloads/Lost Judgment - All Cutscenes Full Movie (2021).f251.webm
[download] 100% of  831.96MiB in 00:00:51 at 16.24MiB/s  
[Merger] Merging formats into "downloads/Lost Judgment - All Cutscenes Full Movie (2021).mp4"
Deleting original file downloads/Lost Judgment - All Cutscenes Full Movie (2021).f299.mp4 (pass -k to keep)
Deleting original file downloads/Lost Judgment - All Cutscenes Full Movie (2021).f251.webm (pass -k to keep)
Video downloaded successfully and saved in: downloads


## Add cookies (optional)

Paste your Netscape-format cookies below (e.g. from [Get cookies.txt LOCALLY](https://github.com/kairi003/Get-cookies.txt-Locally)), then run the cell. The file will be saved and used by the full workflow cell if you set `CONFIG["cookie_file_path"]` to the path shown.

In [ ]:
# Path where the cookie file will be saved (used by the full workflow cell below)
COOKIE_FILE_PATH = "cookies.txt"

# Paste your Netscape-format cookie content between the triple quotes (e.g. export from Get cookies.txt LOCALLY)
COOKIE_CONTENT = """

"""

if COOKIE_CONTENT.strip() and not COOKIE_CONTENT.strip().startswith("# Paste"):
    with open(COOKIE_FILE_PATH, "w", encoding="utf-8", newline="\n") as f:
        f.write(COOKIE_CONTENT.strip())
    print(f"Cookies saved to {COOKIE_FILE_PATH!r}. Set CONFIG['cookie_file_path'] to {COOKIE_FILE_PATH!r} in the full workflow cell below.")
else:
    print("No cookie content to save. Paste cookies between the triple quotes above and run again.")

Cookies saved to 'cookies.txt'. Set CONFIG['cookie_file_path'] to 'cookies.txt' in the full workflow cell below.


## 3. Full workflow: Download → Zip → Upload to Gofile

Download YouTube videos or playlists (with optional cookies), zip the results, and upload to Gofile.io. Supports cookie file path or paste (e.g. from Get cookies.txt LOCALLY).

In [ ]:
import yt_dlp  # pyright: ignore[reportMissingImports]
import os
import re
import shutil
import requests
from typing import Any

# --- Optional config: set USE_CONFIG = True and fill CONFIG to skip interactive prompts ---
USE_CONFIG = True
CONFIG = {
    "video_urls": [

    ],           # e.g. ["https://www.youtube.com/watch?v=...", "https://www.youtube.com/playlist?list=..."]
    "base_save_path": "downloads",
    "cookie_file_path": None,   # Path to Netscape cookie file. Set to "cookies.txt" after running the "Add cookies" cell above, or None to skip.
    "compression": "individual",    # "single" or "individual"
}
# Use cookie file from "Add cookies" cell if it exists
try:
    if "COOKIE_FILE_PATH" in dir() and os.path.isfile(COOKIE_FILE_PATH):
        CONFIG["cookie_file_path"] = COOKIE_FILE_PATH
except NameError:
    pass

def download_youtube_video(video_urls, base_save_path="downloads", cookie_file=None):
    """
    Downloads YouTube videos or playlists in the best available video and audio quality into dynamically named subfolders.

    Args:
        video_urls (list): A list of URLs for YouTube videos or playlists.
        base_save_path (str): The base directory to save the downloaded content.
        cookie_file (str, optional): Path to a Netscape-format cookie file. Defaults to None.
    """
    # Ensure the base save directory exists
    os.makedirs(base_save_path, exist_ok=True)

    for url in video_urls:
        url = url.strip()
        if not url:
            continue

        try:
            # Common options for yt-dlp, including cookie file if provided
            # Use a permissive format and ignoreerrors so info extraction doesn't fail on partial playlists
            common_ydl_opts: dict[str, Any] = {
                'quiet': True,
                'simulate': True,
                'no_warnings': True,  # Suppress warnings about JS runtime
                'format': 'bestvideo*+bestaudio*/bestvideo+bestaudio/best',  # * = any; fallback to single best
                'ignoreerrors': True,  # Allow partial playlists when some videos fail (e.g. format unavailable)
            }
            if cookie_file:
                common_ydl_opts['cookiefile'] = cookie_file

            # Initialize a temporary YoutubeDL instance to extract info without downloading
            print(f"Extracting information for URL: {url}")
            with yt_dlp.YoutubeDL(common_ydl_opts) as ydl_info_extractor:
                info_dict = ydl_info_extractor.extract_info(url, download=False)

            if info_dict is None:
                print(f"Could not extract information for {url}. Skipping.")
                continue

            # Determine folder name based on content type (video or playlist)
            if info_dict.get('_type') == 'playlist':
                folder_name = info_dict.get('title', 'Unknown Playlist')
            else:
                folder_name = info_dict.get('title', 'Unknown Video')

            # Sanitize folder name to remove invalid characters, ensuring folder_name is a string
            sanitized_folder_name = re.sub(r'[\\/:*?"<>|]', '_', str(folder_name))
            current_output_path = os.path.join(base_save_path, sanitized_folder_name)

            # Ensure the specific output directory for this video/playlist exists
            os.makedirs(current_output_path, exist_ok=True)
            print(f"Downloading content for '{folder_name}' into: {current_output_path}")

            # Options for yt-dlp for the current download
            # Robust format with fallbacks; ignoreerrors so one bad video doesn't stop a playlist
            ydl_opts_for_current_download: dict[str, Any] = {
                'format': 'bestvideo*+bestaudio*/bestvideo+bestaudio/best',  # * = any; fallback to single best
                'outtmpl': f'{current_output_path}/%(title)s.%(ext)s',  # Save file format in the specific folder
                'merge_output_format': 'mp4',  # Merge video and audio into MP4 format
                'no_warnings': True,  # Suppress warnings about JS runtime
                'ignoreerrors': True,  # Skip failed videos in a playlist, continue with others
            }
            if cookie_file:
                ydl_opts_for_current_download['cookiefile'] = cookie_file

            # Downloading the video/playlist
            with yt_dlp.YoutubeDL(ydl_opts_for_current_download) as ydl_downloader:
                ydl_downloader.download([url])
            print(f"Content for '{folder_name}' downloaded successfully and saved in: {current_output_path}")

        except Exception as e:
            print(f"An error occurred while processing {url}: {e}")

def compress_downloaded_content(base_save_path, compression_choice):
    """
    Compresses downloaded content into zip archives based on user preference.

    Args:
        base_save_path (str): The base directory where content is downloaded.
        compression_choice (str): 'single' for one zip, 'individual' for multiple zips.

    Returns:
        list: A list of paths to the created zip files.
    """
    zip_files = []
    print(f"Starting compression with choice: {compression_choice}")

    if compression_choice == 'single':
        archive_name = os.path.join(base_save_path, os.path.basename(base_save_path) + '_archive')
        try:
            shutil.make_archive(archive_name, 'zip', base_save_path)
            zip_file_path = archive_name + '.zip'
            zip_files.append(zip_file_path)
            print(f"Created single zip archive: {zip_file_path}")
        except Exception as e:
            print(f"Error creating single zip archive: {e}")
    elif compression_choice == 'individual':
        for item in os.listdir(base_save_path):
            item_path = os.path.join(base_save_path, item)
            if os.path.isdir(item_path):
                archive_name = os.path.join(base_save_path, item)
                try:
                    shutil.make_archive(archive_name, 'zip', item_path)
                    zip_file_path = archive_name + '.zip'
                    zip_files.append(zip_file_path)
                    print(f"Created individual zip archive for {item}: {zip_file_path}")
                except Exception as e:
                    print(f"Error creating zip archive for {item}: {e}")
    return zip_files

def upload_to_gofile(file_path):
    """
    Uploads a file to Gofile.io and returns the shareable download URL.

    Args:
        file_path (str): The path to the file to upload.

    Returns:
        str: The shareable download URL, or None if the upload fails.
    """
    try:
        # 1. Get the best available Gofile.io server
        server_response = requests.get("https://api.gofile.io/servers")
        server_response.raise_for_status() # Raise an exception for HTTP errors
        server_data = server_response.json()

        if server_data['status'] != 'ok' or not server_data['data']['servers']:
            print("Error: Could not get Gofile.io server information.")
            return None

        server_name = server_data['data']['servers'][0]['name']
        upload_url = f"https://{server_name}.gofile.io/uploadFile"
        print(f"Using Gofile.io server: {server_name}")

        # 2. Upload the file
        with open(file_path, 'rb') as f:
            files = {'file': f}
            upload_response = requests.post(upload_url, files=files)
            upload_response.raise_for_status() # Raise an exception for HTTP errors
            upload_data = upload_response.json()

        if upload_data['status'] == 'ok':
            download_page = upload_data['data']['downloadPage']
            print(f"Successfully uploaded {os.path.basename(file_path)}. Download page: {download_page}")
            return download_page
        else:
            print(f"Gofile.io upload failed for {os.path.basename(file_path)}: {upload_data.get('status', 'Unknown error')}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Network error during Gofile.io upload for {os.path.basename(file_path)}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during Gofile.io upload for {os.path.basename(file_path)}: {e}")
        return None

if __name__ == "__main__":
    if USE_CONFIG and CONFIG.get("video_urls"):
        video_urls = CONFIG["video_urls"]
        base_save_path = CONFIG.get("base_save_path", "downloads") or "downloads"
        cookie_file_path = CONFIG.get("cookie_file_path") or None
        if cookie_file_path and not os.path.isfile(cookie_file_path):
            cookie_file_path = None  # Don't pass non-existent path to yt-dlp
        compression_choice = CONFIG.get("compression", "single") or "single"
    else:
        all_urls_input = []
        print("Enter YouTube video or playlist URLs (one per line, press Enter on an empty line to finish):")
        while True:
            url_input = input().strip()
            if not url_input:
                break
            split_urls = re.split(r'[\n,]+', url_input)
            all_urls_input.extend([url.strip() for url in split_urls if url.strip()])
        video_urls = [url for url in all_urls_input if url]
        if not video_urls:
            print("No URLs provided. Exiting.")
            raise SystemExit(0)
        base_save_path = input("Enter the base save path (or press Enter for default 'downloads'): ").strip() or "downloads"
        cookie_file_path = input("Enter path to cookie file (Netscape format, e.g. from Get cookies.txt LOCALLY; press Enter to skip): ").strip() or None
        compression_choice = None
        while compression_choice not in ['single', 'individual']:
            choice_input = input("Create a 'single' zip or 'individual' zips per subfolder? (default: single): ").strip().lower()
            if not choice_input or choice_input == 'single':
                compression_choice = 'single'
            elif choice_input == 'individual':
                compression_choice = 'individual'
            else:
                print("Invalid choice. Enter 'single' or 'individual'.")

    print(f"Compression: {compression_choice}")
    download_youtube_video(video_urls, base_save_path, cookie_file_path)
    compressed_files = compress_downloaded_content(base_save_path, compression_choice)
    print(f"Compressed files: {compressed_files}")
    gofile_links = []
    print("\n--- Uploading files to Gofile.io ---")
    for zip_file in compressed_files:
        link = upload_to_gofile(zip_file)
        if link:
            gofile_links.append(link)
    if gofile_links:
        print("\n--- Gofile.io Download Links ---")
        for i, link in enumerate(gofile_links):
            print(f"Link {i+1}: {link}")
    else:
        print("No files were successfully uploaded to Gofile.io.")

Compression: individual
Extracting information for URL: https://www.youtube.com/playlist?list=PL4_bo90i-4GJfr_KaIR6nJpnhOm7twiqQ


ERROR: [youtube] efigiluZEHA: Requested format is not available. Use --list-formats for a list of available formats


An error occurred while processing https://www.youtube.com/playlist?list=PL4_bo90i-4GJfr_KaIR6nJpnhOm7twiqQ: ERROR: [youtube] efigiluZEHA: Requested format is not available. Use --list-formats for a list of available formats
Extracting information for URL: https://www.youtube.com/playlist?list=PL4_bo90i-4GJWh4JiuPD9t8I93K79vz1b


ERROR: [youtube] RiMga7zpVfI: Requested format is not available. Use --list-formats for a list of available formats


An error occurred while processing https://www.youtube.com/playlist?list=PL4_bo90i-4GJWh4JiuPD9t8I93K79vz1b: ERROR: [youtube] RiMga7zpVfI: Requested format is not available. Use --list-formats for a list of available formats
Extracting information for URL: https://www.youtube.com/playlist?list=PL4_bo90i-4GJAI36WoC2hymON9bnYvmbj


...

An error occurred while processing https://www.youtube.com/playlist?list=PL4_bo90i-4GKattvwei-Pj5yLGjypotdo: ERROR: [youtube] m5MSWv9nJfM: Requested format is not available. Use --list-formats for a list of available formats
Extracting information for URL: https://www.youtube.com/playlist?list=PL4_bo90i-4GJlvus_NPGaG8_-jAyb4w7K


ERROR: [youtube] Ur-JBlBPt6I: Requested format is not available. Use --list-formats for a list of available formats


An error occurred while processing https://www.youtube.com/playlist?list=PL4_bo90i-4GJlvus_NPGaG8_-jAyb4w7K: ERROR: [youtube] Ur-JBlBPt6I: Requested format is not available. Use --list-formats for a list of available formats
Extracting information for URL: https://www.youtube.com/playlist?list=PL4_bo90i-4GI7-1qKIRD36RiqXQgm9QoA


ERROR: [youtube] NJ-JcDcff8o: Requested format is not available. Use --list-formats for a list of available formats


An error occurred while processing https://www.youtube.com/playlist?list=PL4_bo90i-4GI7-1qKIRD36RiqXQgm9QoA: ERROR: [youtube] NJ-JcDcff8o: Requested format is not available. Use --list-formats for a list of available formats
Starting compression with choice: individual
Compressed files: []

--- Uploading files to Gofile.io ---
No files were successfully uploaded to Gofile.io.
